In [1]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import gzip, json
from Bio import SeqIO
import scipy.sparse as sp
from collections import Counter
import numpy as np
import pandas as pd
fasta_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/reads.fasta.gz'
paf_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/alignment.paf.gz'
with gzip.open(paf_gz_file, "rt") as file:
    max_values = {}  
    for row in file:  
        columns = row.strip().split('\t') 
        query_id = columns[0]  
        match_bases = int(columns[9]) 
        max_values[query_id] = columns 
        if query_id in max_values:  
            if match_bases > int(max_values[query_id][9]):  
                max_values[match_bases] = columns
        else:  
            continue

In [2]:
import sys
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy as sp
from typing import Mapping  
import mmh3
from itertools import chain  
import sharedmem

In [3]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from nearest_neighbors import HNSW


/home/miaocj/docker_dir/kNN-overlap-finder/scripts/../lib


In [14]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  
import random
max_n_neighbors = 20
MAX_SAMPLE_SIZE = int(1e9)
k_values = np.arange(2, max_n_neighbors + 1)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)


In [5]:
import secrets
random.seed(12)  # 这里的10可以替换为你希望的种子值  
random_number = secrets.token_bytes(128)
random_integer = int.from_bytes(random_number, byteorder='big')  
binary_array = format(random_integer, '01024b')  

In [6]:
npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/read_features.json.gz"

meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

read_ids = np.array(list(read_features))

In [7]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

(12926, 324674, 25276, 2)

In [87]:
##法一：用secret函数 kmer_index 直接作为种子
def _hash(kmer_index: int) -> np.ndarray:
    random.seed(kmer_index)
    random_number = secrets.token_bytes(400)
    random_integer = int.from_bytes(random_number, byteorder='big')  
    binary_array = format(random_integer, '03200b')
    hash_array = list(binary_array)
    return hash_array
def _get_table(
    kmer_num: list) -> Mapping[int,list]:
    hash_table = np.empty((kmer_num,3200),dtype=np.int8) 
    for kmer_index in range(kmer_num):
        hash_array = _hash(kmer_index)
        hash_table[kmer_index,:]=hash_array
    hash_table = np.where(hash_table == 0, -1, 1) 
    return hash_table
hash_table = _get_table(feature_matrix.shape[1])


In [8]:
##法二：用secret函数 kmer_index的哈希值作为种子
def _hash(kmer_index: int) -> np.ndarray:
    hash_value = mmh3.hash(str(kmer_index))
    random.seed(hash_value)
    random_number = secrets.token_bytes(400)
    random_integer = int.from_bytes(random_number, byteorder='big')  
    binary_array = format(random_integer, '03200b')
    hash_array = list(binary_array)
    return hash_array
def _get_table(
    kmer_num: list) -> Mapping[int,list]:
    hash_table = np.empty((kmer_num,3200),dtype=np.int8) 
    for kmer_index in range(kmer_num):
        hash_array = _hash(kmer_index) 
        hash_table[kmer_index,:]=hash_array
    hash_table = np.where(hash_table == 0, -1, 1) 
    return hash_table
hash_table = _get_table(feature_matrix.shape[1])

In [79]:
##法三：用random模块 哈希值作为种子
def _hash(kmer_index: int) -> np.ndarray:
    hash_value = mmh3.hash(str(kmer_index))
    random.seed(hash_value)
    hash_array = list(bin(random.getrandbits(3200))[2:].zfill(3200))
    return hash_array
def _get_table(
    kmer_num: list) -> Mapping[int,list]:
    hash_table = np.empty((kmer_num,3200),dtype=np.int8) 
    for kmer_index in range(kmer_num):
        hash_array = _hash(kmer_index)
        hash_table[kmer_index,:]=hash_array
    hash_table = np.where(hash_table == 0, -1, 1) 
    return hash_table
hash_table = _get_table(feature_matrix.shape[1])

In [ ]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int8) 
    hash_array = np.where(hash_array == 0, -1, 1)
    return hash_array
def _get_table(
    kmer_num: list,  
    *,
    seed: int,
    repeat=100) -> Mapping[int,list]:  
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
    hash_table = np.empty((kmer_num,repeat,32),dtype=np.int8)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag,:]=_hash(kmer_index, seed=seed)
            new_hash_table=np.reshape(hash_table,(kmer_num,32*repeat))
    return new_hash_table
hash_table = _get_table(feature_matrix.shape[1])

In [7]:
new = np.load('/home/miaocj/docker_dir/data/metagenome/hash_table.npz')
hash_table2 = new['arr_0'][:feature_matrix.shape[1],:]
hash_table3 = hash_table2.astype(np.int8)
hash_table4 = np.where(hash_table3 == 0, -1,1)

In [13]:
new['arr_0'].shape

(15055916, 1920)

In [85]:
def _get_simhash(
    read_features: list,  
    hash_table) -> Mapping[int,list]:  
    all_read_simhash = []

    for read_kmer in read_features.values():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        simhash = np.where(one_read_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)
    return reads_simhash_array
reads_simhash_array = _get_simhash(read_features,hash_table)

In [80]:
simhash = feature_matrix@hash_table
reads_simhash_array = np.where(simhash > 0, 1, 0)

In [81]:

nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(reads_simhash_array)  
indices = nbrs.kneighbors(reads_simhash_array,return_distance=False)
nbr_indices = indices[:, 1:]

df_rows = []
for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'method3', "n_neighbors": k, "repeat_time": 1,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)
#df_tfidf.to_csv('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/human/HLA/ONT_R9/kmer_k16/SimHash_None_None_overlap_stat.tsv',sep='\t')

In [82]:
##法一 ：哈希长度为1028
df
df.to_csv('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/human/HLA/ONT_R9/kmer_k16/SimHash_method3_overlap_stat.tsv',sep='\t')

In [86]:
df

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,component_sizes
0,method3,2,1,0.991711,0.325411,0.061172,0.257834,120,0.009284,176,"[648, 570, 444, 404, 364, 338, 330, 313, 295, ..."
1,method3,3,1,0.990196,0.288545,0.089588,0.335338,83,0.006421,2250,"[2869, 2481, 2250, 1288, 1203, 600, 599, 486, ..."
2,method3,4,1,0.988828,0.263292,0.117490,0.401844,63,0.004874,3956,"[6450, 3956, 2494, 2, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,method3,5,1,0.987573,0.242501,0.144899,0.457034,50,0.003868,6450,"[6452, 6450, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,method3,6,1,0.986382,0.224787,0.172009,0.503521,47,0.003636,6451,"[6453, 6451, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,method3,7,1,0.985119,0.211648,0.198596,0.548069,44,0.003404,6452,"[6453, 6452, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,method3,8,1,0.984119,0.199582,0.224832,0.585694,42,0.003249,6452,"[6453, 6452, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,method3,9,1,0.982969,0.190071,0.250827,0.623002,39,0.003017,6453,"[6453, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,method3,10,1,0.981669,0.181148,0.276271,0.654850,38,0.002940,6453,"[6453, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,method3,11,1,0.980382,0.173465,0.301533,0.685314,37,0.002862,6453,"[6453, 6453, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
ref_reads_simhash_array = get_simhash(ref_read_features,hash_table)

nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric=hamming_distance)
nbrs.fit(ref_reads_simhash_array)
indices = nbrs.kneighbors(que_reads_simhash_array,return_distance=False)
print("done\nevaluates")

In [35]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int8) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_table(
    read_features: list,  
    feature_matrix: list,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  

    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat,32),dtype=np.int8)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag,:]=_hash(kmer_index, seed=seed)
            new_hash_table=np.reshape(hash_table,(kmer_num,3200))
    return new_hash_table

def _get_simhash(
    read_features: list,  
    feature_matrix: list,
    hash_table,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    all_read_simhash = []
    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        weighted_hash_table = hash_table*idf[:, np.newaxis]
    else:
        weighted_hash_table = hash_table
    if tf == False:
        read_features = {k:list(set(v)) for k,v in read_features.items()}

    for read_kmer in read_features.values():
        one_read_hash = np.sum(weighted_hash_table[read_kmer,:],axis=0)
        simhash = np.where(one_read_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)

    return reads_simhash_array 

hash_table = _get_table(read_features,feature_matrix,seed = 15232,tf = False,idf=False,repeat=100)
print("1")
reads_simhash_array = _get_simhash(read_features,feature_matrix,hash_table,seed = 15232,tf = True,idf=False,repeat=100)

In [6]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int8) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  
def mp_get_hashtable(  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    processes:int,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat,32),dtype=np.int8) 

    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            seed = hash_seeds[i]
            result = np.empty((kmer_num,32), dtype=np.int8) 
            for kmer_index in range(kmer_num):
                result[kmer_index,:]=_hash(kmer_index, seed=seed)
            return i,result

        def reduce(i, result):
            hash_table[:,i,:] = result            

        pool.map(work, range(repeat), reduce=reduce)
    return hash_table
hash_table = mp_get_hashtable(read_features,feature_matrix,15232,10)

In [19]:
one_read_hash = np.sum(hash_table[list(read_features.values())[0],:,:],axis=0)
hash_table

In [26]:
hash_table.shape

In [34]:
kmer_num = feature_matrix.shape[1]
hash_table=np.reshape(hash_table,(kmer_num,3200))

In [30]:
len(new_hash[1])

In [18]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

In [19]:
read_ids = np.array(list(read_features))

In [20]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def mp_get_hashtable(  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    processes:int,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object) 

    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            seed = hash_seeds[i]
            result = np.empty(kmer_num, dtype=object) 
            for kmer_index in range(kmer_num):
                result[kmer_index]=_hash(kmer_index, seed=seed)
            return i,result

        def reduce(i, result):
            hash_table[:,i] = result            

        pool.map(work, range(repeat), reduce=reduce)
    return hash_table

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

## multi-process calculate simhash value
def mp_get_simhash(
    read_features: list,  
    feature_matrix: list,
    hash_table,
    tf:bool,
    idf:bool,
    processes:int,):
    
    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        hash_table = hash_table*idf[:, np.newaxis]
    if tf == False:
        read_features = {k:list(set(v)) for k,v in read_features.items()} 

       
    reads_simhash_array = np.empty((len(read_features),hash_table.shape[1]*32),dtype=object)
    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            read_kmer = list(read_features.values())[i]
            one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
            conc_hash = np.concatenate(one_read_hash)
            result = np.where(conc_hash > 0, 1, 0)
            return i,result

        def reduce(i, result):
            reads_simhash_array[i,:] = result            
        pool.map(work, range(len(read_features)), reduce=reduce)
    return reads_simhash_array

In [21]:
hash_table = mp_get_hashtable(feature_matrix,repeat =100, seed = 4829,processes=12)
print('hashtable established')

In [16]:
df_tfidf

In [111]:
df_tfidf

In [ ]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3

for repeat in [100]:
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829,tf = True,idf = True)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)

In [ ]:
repeat = 100

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

all_read_simhash = []
rng = np.random.default_rng(4829)  
hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)
kmer_num = feature_matrix.shape[1]
hash_table = np.empty((kmer_num,repeat),dtype=object)  
for flag,seed in enumerate(hash_seeds):
    print(flag)
    kmer_hash_indice = {} 
    for kmer_index in range(kmer_num):
        hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)
new_hash = hash_table*idf[:, np.newaxis]

for read_ind,read_kmer in read_features.items():
    one_read_hash = np.sum(new_hash[read_kmer,:],axis=0)
    conc_hash = np.concatenate(one_read_hash)
    simhash = np.where(conc_hash > 0, 1, 0)
    all_read_simhash.append(simhash)
reads_simhash_array = np.array(all_read_simhash)

In [ ]:
reads_simhash_array

In [ ]:
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(reads_simhash_array)  
indices = nbrs.kneighbors(reads_simhash_array,return_distance=False)
nbr_indices = indices[:, 1:]

read_ids = np.array(list(read_features))
k_values = np.arange(2, max_n_neighbors + 1)
df_rows = []

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
##带tf idf 初始版本
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    tf:bool,
    idf:bool,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    all_read_simhash = []
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)

    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        hash_table = hash_table*idf[:, np.newaxis]
    if tf == False:
        read_features = {k:set(v) for k,v in read_features}

    for read_kmer in read_features.values():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        conc_hash = np.concatenate(one_read_hash)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)
  
    return reads_simhash_array 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

In [ ]:
new[3][1]

In [ ]:
df

In [ ]:
##32位，70个repeat
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  

    hash_seeds = np.array(range(repeat))
    kmer_num = feature_matrix.shape[1]
    kmer_index = np.array(range(kmer_num))

    table = []
    for seed in hash_seeds:
        prime1 = 2654435761  # A large prime number
        prime2 = 0x27d4eb2d  # Another large prime, often used in hashing
        hash_value = (kmer_index * prime1) ^ (seed * prime2)
        hash_t = hash_value % (2**32)
        binary_matrix = np.vectorize(np.binary_repr)(hash_t, width=32)
        one_repear_table = np.array([list(row) for row in binary_matrix.flatten()])
        table.append(one_repear_table.astype(int))
    hash_table = np.hstack(table)

    all_read_simhash = []
    for read_ind,read_kmer in read_features.items():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    concat_simhash = np.array(all_read_simhash)
    return concat_simhash 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

dfs = []
for repeat in range(20,110,10):
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graphs[k] = graph

    df_rows = []
    for k in k_values:
        graph = graphs[k]
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    dfs.append(df)
    
new = pd.concat(dfs)
new.to_csv('/home/miaocj/docker_dir/test_simhash.csv',sep='\t')

In [ ]:
new

In [ ]:
meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

feature_weights = {i: 1 for i in range(feature_matrix.shape[1])}

In [ ]:
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graphs[k] = graph

In [ ]:
df_rows = []

for k in k_values:
    graph = graphs[k]
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, 
                 **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
##70
df

In [ ]:
##40
df

In [ ]:
##20
df

测试hash方法

In [ ]:
import hashlib

def simhash(features):
  
  # Generate a hash for each feature
  hashes = [hashlib.sha1(feature).hexdigest() for feature in features]
  
  # Combine the feature hashes to produce the final simhash
  concatenated_hash = ''.join(hashes)
  simhash = hashlib.sha1(concatenated_hash).hexdigest()
  
  return simhash

In [ ]:
x

In [ ]:
pip install xxhash

In [ ]:
import xxhash
x = np.random.rand(1024 * 1024 * 16)
h = xxhash.xxh64()
h.update(x); h.intdigest(); h.reset()

In [ ]:
x

In [ ]:
## 64位， 40个repeat
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:064b}".format(hash_value & 0xFFFFFFFFFFFFFFFF)
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]):  
            kmer_index_str = str(kmer_index)  
            hash_list = _hash(kmer_index_str, seed=s)  
            kmer_hash_indice[kmer_index] = hash_list  
  
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 40, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]


In [ ]:
rng = np.random.default_rng(seed)
beta = rng.uniform(0, 1, (feature_count, dimension_count))
x = rng.uniform(0, 1, (feature_count, dimension_count))
u1 = rng.uniform(0, 1, (feature_count, dimension_count))
u2 = rng.uniform(0, 1, (feature_count, dimension_count))

for j_sample in range(0, sample_count):
    feature_indices = sparse.find(data[:, j_sample] > 0)[0]
    gamma = -np.log(np.multiply(u1[feature_indices, :], u2[feature_indices, :]))
    t_matrix = np.floor(
        np.divide(
            matlib.repmat(
                np.log(data[feature_indices, j_sample].todense()),
                1,
                dimension_count,
            ),
            gamma,
        )
        + beta[feature_indices, :]
    )
    y_matrix = np.exp(np.multiply(gamma, t_matrix - beta[feature_indices, :]))
    a_matrix = np.divide(
        -np.log(x[feature_indices, :]),
        np.divide(y_matrix, u1[feature_indices, :]),
    )

In [ ]:
rng = np.random.default_rng(12)

In [ ]:
##建立hash矩阵
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    hash_table = np.empty((repeat,feature_matrix.shape[1]))  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]): 
            hash_table[s,kmer_index] = _hash(kmer_index, seed=s)
    for 
         
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 70, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]